<h1>Applied Data Science Capstone Project</h1>

<p>This notebook was put together for the IBM Data Science certificate capstone project. This first version will describe the business problem of the capstone and related data to solve problem.</p>

<h3>The business idea</h3>
<p>I have a friend who is considering on moving to the Capital area in Finland who has never lived in the area and is unaware of the neighbourhoods. Coming from rural areas he's a mainly concerned about about finding a location to live where: a) The pricing is not too expensive b) the available public transport to his new workplace in Otaniemi would be relative fast (45 - 60min max by public transport) c) the place is not too restless and unsafe d) there would be parks, gardens, cafeterias etc for unwinding on free time.</p>

<p>As the capital area is relatively large there are a lot of different neighbourhoods to choose from. There are certain "bad" neighbourhoods where the crimes are more common. These neighbourhoods usually also are either located in the transit hubs or have more bars etc in the areas. The speed of the public transportation also varies quite a bit depending on how many transit changes are required and how close the start and end points are to the main hubs. While the prices in the whole capital area are quite high in relation to other areas there is also a lot of difference across the neighbourhoods. The more quiet areas with more parks and less density of population are usually also quite far in terms of public transport.</p>

<p>It would be interesting to have a service that would recommend neighbourhoods based on this criteria when planning to purchase an apartment. Personally I spent around 8 months hunting for an apartment and trying to figure out the neighbourhoods. </p>

<h3>The data</h3>
<p>Solving this problem requires quite a few separate services to obtain the data. The main types of data that we will need to solve the problem are: location data with different venues, data of the crime rates, dataset or a service to build the data set from the distances and estimated travel, average prices data for the neighbourhoods.</p>

<p><b>Location data</b> will come from FourSquare the data will be used to evaluate the "restlessness" or how lively the neighbourhood is and to obtain the most common types of venues and trying to make some recommendations based on the most desired venue types.</p>

<p>Since getting it from a primary source per neighbourhoods is quite hard <b>the crime data</b> will come from infogram that has been published in a newspaper and been built from data coming from Statistics Finland and the police. The data will be used to visualise the crime occurances on a map and as one basis for evaluating the optimum neighbourhood.</p>

<p><b>The average prices data</b> will be acquired from Statistics Finland and the obtained csv will have to be wrangled a bit to produce the average price of the apartments. The data </p>

<p><b>The public transport data</b> will be acquired from HSL which is the public transport provider for the whole capital area. The collected data will be used to rank the neighbourhoods based on the speed of the transport and to restrict neighbourhoods that are too far from the home to work transit time requirements.</p>


In [68]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import folium
import requests
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [1]:
https://fi.wikipedia.org/wiki/Luettelo_Suomen_postinumeroista_kunnittain

SyntaxError: invalid syntax (<ipython-input-1-439431bcc326>, line 1)

In [1]:
https://fi.wikipedia.org/wiki/Luettelo_Suomen_postinumeroista_kunnittain

SyntaxError: invalid syntax (<ipython-input-1-439431bcc326>, line 1)

In [2]:
#import required dependencies
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [2]:
#import required dependencies
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [109]:
address = 'Espoo, Uusimaa, Finland'

geolocator = Nominatim(user_agent="espoo_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Espoo is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Espoo is 60.2241869, 24.6603626.


In [109]:
address = 'Espoo, Uusimaa, Finland'

geolocator = Nominatim(user_agent="espoo_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Espoo is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Espoo is 60.2241869, 24.6603626.


In [110]:
# Generate at new folium map from latitude and longitude values
map_espoo = folium.Map(location=[latitude, longitude], zoom_start=11)

map_espoo

In [110]:
# Generate at new folium map from latitude and longitude values
map_espoo = folium.Map(location=[latitude, longitude], zoom_start=11)

map_espoo

In [3]:
# Set url
url = "https://fi.wikipedia.org/wiki/Luettelo_Suomen_postinumeroista_kunnittain"
# Query the page html
page = requests.get(url)
# Import the page into bs4
soup = BeautifulSoup(page.text, 'html.parser')

In [3]:
# Set url
url = "https://fi.wikipedia.org/wiki/Luettelo_Suomen_postinumeroista_kunnittain"
# Query the page html
page = requests.get(url)
# Import the page into bs4
soup = BeautifulSoup(page.text, 'html.parser')

In [114]:
#print(soup)
espoo_neighbourhoods = pd.DataFrame(columns=['PostalCode', 'Neighbourhood', 'Latitude', 'Longitude'])

postal_codes = soup.find_all('li')
for code in postal_codes:
    row = code.get_text()
    postal_code = re.findall(r'^[0-9]*', row)[0];
    if('Espoo / Esbo' in row and postal_code != ''):
        neighbourhoods = row.split('Espoo / Esbo –')[1].strip()
#        print(postal_code, neighbourhoods)
        
        address = postal_code + ', Espoo, Uusimaa, Finland'

        geolocator = Nominatim(user_agent="espoo_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
#        print(postal_code, latitude, longitude)
        espoo_neighbourhoods = espoo_neighbourhoods.append(
            { 'PostalCode': postal_code, 'Neighbourhood': neighbourhoods, 'Latitude': latitude, 'Longitude': longitude },
            ignore_index=True)


In [114]:
#print(soup)
espoo_neighbourhoods = pd.DataFrame(columns=['PostalCode', 'Neighbourhood', 'Latitude', 'Longitude'])

postal_codes = soup.find_all('li')
for code in postal_codes:
    row = code.get_text()
    postal_code = re.findall(r'^[0-9]*', row)[0];
    if('Espoo / Esbo' in row and postal_code != ''):
        neighbourhoods = row.split('Espoo / Esbo –')[1].strip()
#        print(postal_code, neighbourhoods)
        
        address = postal_code + ', Espoo, Uusimaa, Finland'

        geolocator = Nominatim(user_agent="espoo_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
#        print(postal_code, latitude, longitude)
        espoo_neighbourhoods = espoo_neighbourhoods.append(
            { 'PostalCode': postal_code, 'Neighbourhood': neighbourhoods, 'Latitude': latitude, 'Longitude': longitude },
            ignore_index=True)


In [115]:
espoo_neighbourhoods

,PostalCode,Neighbourhood,Latitude,Longitude
0,02100,"Tapiola – Heikintori, U-talo",60.177763,24.803189
1,02110,Tapiola – Itäranta,60.178228,24.816331
2,02120,"Tapiola – Suvikumpu, Hakalehto",60.172990,24.790880
3,02130,"Pohjois-Tapiola – Koivumankkaa, Pyhäristi",60.187425,24.793208
4,02140,"Laajalahti – Friisinmäki, Laajaranta",60.202770,24.805547
5,02150,"Otaniemi – Keilaniemi, Servinniemi, Teekkariky...",60.184174,24.829973
6,02160,Westend,60.167467,24.801453
7,02170,Haukilahti,60.159280,24.775274
8,02180,"Mankkaa – Seilimäki, Klovi, Taavinkylä",60.197960,24.761569
9,02200,"Henttaa, Niittykumpu – Piispankylä, Olarinluom...",60.172275,24.763034


In [115]:
espoo_neighbourhoods

,PostalCode,Neighbourhood,Latitude,Longitude
0,02100,"Tapiola – Heikintori, U-talo",60.177763,24.803189
1,02110,Tapiola – Itäranta,60.178228,24.816331
2,02120,"Tapiola – Suvikumpu, Hakalehto",60.172990,24.790880
3,02130,"Pohjois-Tapiola – Koivumankkaa, Pyhäristi",60.187425,24.793208
4,02140,"Laajalahti – Friisinmäki, Laajaranta",60.202770,24.805547
5,02150,"Otaniemi – Keilaniemi, Servinniemi, Teekkariky...",60.184174,24.829973
6,02160,Westend,60.167467,24.801453
7,02170,Haukilahti,60.159280,24.775274
8,02180,"Mankkaa – Seilimäki, Klovi, Taavinkylä",60.197960,24.761569
9,02200,"Henttaa, Niittykumpu – Piispankylä, Olarinluom...",60.172275,24.763034
